This script (in Julia) computes spectral features of neural data:
Spike-field measures:
    Phase-locking values
    Pairwise-phase consistency
    Spike-field coherence
Field-field measures:
    Coherence
    Phase-locking values
    Pairwise-phase consistency
    Canonical coherence (SVD method)
Jack-knife esimated variances are also computed except for Canonical Coherence.

In [3]:
############################## 
# Load modules 
##############################

using MAT, Synchrony  # DSP, Distributions, HypothesisTests

In [4]:
############################## 
# Declare helper functions 
##############################

function fileHandle(session::ASCIIString)
    ## Set up file locations
    fileHandle = Dict()
    fileHandle["dataDir"] = "C:\\Users\\jian\\Documents\\offlineResult"
    fileHandle["session"] = session
    fileHandle["suDir"]   = "$(session)-SPK-SU"
    fileHandle["lfpDir"]  = "$(session)-LFP"
    fileHandle["suFileNamePrefix"] = "SPK-SUDelayOnsetTime[-1350  1650]-"
    fileHandle["fileFormat"] = ".mat"
    fileHandle["infoSummary"] = "summary"    
    fileHandle["lfpFileNamePrefix"] = "LFP-Freq[1  500]-DelayOnsetTime[-1350  1650]-"
    return fileHandle
end

function loadLFP(session::ASCIIString, ch::Int64, tar::Int64)
    ## Load LFP file
    file = fileHandle(session)
    selectChFile = join([file["lfpFileNamePrefix"],"ch$(ch)-tar$(tar)",
        file["fileFormat"]])
    lfpFile =joinpath(file["dataDir"],file["session"],file["lfpDir"],selectChFile)
    lfpData = matread(lfpFile)
    return lfpData["lfp"]
end

function loadSU(session::ASCIIString, ch::Int64, tar::Int64, unit::Int64)
    ## Load SPK file
    selectSUUnitFile = join([file["suFileNamePrefix"],"ch$(ch)-unit$(unit)-tar$(tar)",
        file["fileFormat"]])
    suFile = joinpath(file["dataDir"],file["session"],file["suDir"],selectSUUnitFile)
    suData = matread(suFile)
    return suData["spikeCount"]
end

function getPPC(angles)
## Computer PPC using Simon Kornblith's simplified equation , p-value for Rayleigh's test of uniformity
    PPC = 1 - var(exp(im*angles))
    rStat = RayleighTest(angles)
    pVal = pvalue(rStat) 
    return PPC, rStat, pVal
end

## set up frequency bands of interest
freqBands = Dict(
    "delta" => [1 4], 
    "theta" => [4 8],
    "alpha" => [8 12], 
    "beta" =>[12 30],
    "gamma" => [30 80])

## Epoch indices
Epochs = Dict(
"fixation" => collect((-350-450+1):1:(-350)) + 1350, #check with Scott(saw some 450 ms instead of 500ms)
"target" => collect((-350+1):1:0) + 1350,
"delay" => collect(0+1:1:750) + 1350,
"response" => collect(750+1:1:(750+400)) + 1350)



Dict{ASCIIString,Array{Int64,1}} with 4 entries:
  "response" => [2101,2102,2103,2104,2105,2106,2107,2108,2109,2110  …  2491,249…
  "fixation" => [551,552,553,554,555,556,557,558,559,560  …  991,992,993,994,99…
  "delay"    => [1351,1352,1353,1354,1355,1356,1357,1358,1359,1360  …  2091,209…
  "target"   => [1001,1002,1003,1004,1005,1006,1007,1008,1009,1010  …  1341,134…

In [1]:
############################## 
#Set session/tar/unit, etc 
##############################

# TODO : drop down menu to choose channel/tar/unit 
session = "JS20131010" #"CS20120918" #"CS20120918" #
ch = 1
tar = 1
unit = 0

##############################
# Set analysis parameters
##############################
nfft = 512  # number of samples to use for fft
step = 50   # window of nfft points stepped every #step samples
stsStep = 50
fRange = 1:80  # number of frequency points to keep.

println("done")

done


In [5]:
##############################
# Load LFP, compute multitaper
##############################

#for session in collect(sessions)
file = fileHandle(session)
infoFile = joinpath(file["dataDir"],file["session"],file["lfpDir"],
    join([file["lfpFileNamePrefix"], file["infoSummary"], file["fileFormat"]],""))
infoSummary = matread(infoFile)
lfpChs = Array{Int64}(infoSummary["BCIinfo"]["lfpChs"])
println("done")

done


In [6]:
## Get spiking channel meta info
suInfoFile = joinpath(file["dataDir"],file["session"],file["suDir"], 
    join([file["suFileNamePrefix"],file["infoSummary"],file["fileFormat"]], ""))
suInfoSummary = matread(suInfoFile)
spkChs = Array{Int64}(suInfoSummary["BCIinfo"]["spkChs"])
chNumUnits = Array{Int64}(suInfoSummary["BCIinfo"]["chNumUnits"])
println(length(spkChs)," channels contain spikes")

38 channels contain spikes


In [7]:
for tar = 1:6

    println("Processing target ", tar)
    ## Data originally in trials x channels
    ## Shape data into samples x channels x trials for multitaper() (Synchrony.jl)

    ## Initialize data structure
    (a,b) = size(loadLFP(session,1,tar))
    lfpTemp = Array{Float64}(length(lfpChs),a,b)
    suTemp = Array{Int64}(sum(chNumUnits[spkChs]),a,b)
    

    ## helper variables

    suLFPTemp = Array{Float64}(sum(chNumUnits[spkChs]),a,b)
    
    ## Load data into structure
    unitCount = 1
    println("Loading lfps for session: $session, tar: $tar")
    @time for ch in collect(lfpChs)
        lfpTemp[ch,:,:] = loadLFP(session,ch,tar)
        if in(ch, spkChs)
            #println("Channel ",ch," contains ",chNumUnits[ch]," units")
            for unit in 1:chNumUnits[ch]
                #println("Loading unit $unitCount")
                suTemp[unitCount,:,:] = loadSU(session, ch, tar, unit-1) # nTrials x nSamples
                suLFPTemp[unitCount,:,:] = lfpTemp[ch,:,:]
                #println(sum(sum(suTemp[unitCount,:,:])))
                unitCount +=1
            end
        end
    end
    println("Done loading data")
    println( sum(sum(sum(suTemp)))," spikes total")

    ## Permute data dimensions -> samples x channels x trials
    lfpAllChns = permutedims(lfpTemp,[3,1,2])
    suAllChns = permutedims(suTemp,[3,1,2])
    suLFPDup = permutedims(suLFPTemp,[3,1,2])

    ## Get trial average firing rate
    suAllChnsTrlMean = mean(suAllChns, 3)
    suAllChnsTrlStd = std(suAllChns, 3)
    
    ## Clear temp data. Julia does not free declared variables from memory.
    lfpTemp = []
    suTemp = []   
    suLFPTemp = []

    #println(size(suAllChns))

    ## Computer multitaper 

    spectralFileName = joinpath(file["dataDir"],file["session"],file["lfpDir"],
    join([file["lfpFileNamePrefix"], "tar$(tar)-spectral", file["fileFormat"]],""))

    ch2ch = allpairs(length(lfpChs))
    f = collect(frequencies(nfft, 1000)[1][fRange])

    ## indices of data segments
    a = 1:step:size(lfpAllChns,1)-nfft
    b = nfft:step:size(lfpAllChns,1)

    ## Init data structure
    ## For LFP signals
    ps = Array{Float64}(length(f), length(lfpChs),length(a))
    #xs = Array{Complex{Float64}}(length(f), size(ch2ch,2),length(a))
    #coherency = Array{Complex{Float64}}(length(f), size(ch2ch,2),length(a))
    co = Array{Float64}(length(f), size(ch2ch,2),length(a))
    plv = Array{Float64}(length(f), size(ch2ch,2),length(a))
    ppc = Array{Float64}(length(f), size(ch2ch,2),length(a))
    plv_bias = Array{Float64}(length(f), size(ch2ch,2),length(a))
    plv_variance = Array{Float64}(length(f), size(ch2ch,2),length(a))
    coh_bias = Array{Float64}(length(f), size(ch2ch,2),length(a))
    coh_variance = Array{Float64}(length(f), size(ch2ch,2),length(a))

    pli2unbiased = Array{Float64}(length(f), size(ch2ch,2),length(a))
    wpli = Array{Float64}(length(f), size(ch2ch,2),length(a))
    wpli2debiased = Array{Float64}(length(f), size(ch2ch,2),length(a))
    
    ## For spike signals 
    ## create mask for spiketriggeredspectrum and myPPC
    epochMask = zeros(Int64,size(suLFPDup))
    epochMask[Epochs["fixation"][1] : Epochs["response"][end],:,:] += 1
    vecIndex=find(suAllChns.*epochMask.==1)
    (sampleInd, unitInd, trialInd) = ind2sub(size(suAllChns), vecIndex)

    ## Spike-triggered spectrum
    println("Computing spike-triggered-spectrum")
    sts = spiketriggeredspectrum(vecIndex, vec(suLFPDup), -255:256, freqrange = fRange )
    sts_phase = angle(sts)
    
    

    ## Set up windows for sts
    stsWindows = Epochs["fixation"][1]:step:Epochs["response"][end]
    ## Spike-field coherence
    sfc = Array{Float64}(length(f), sum(chNumUnits[spkChs]),length(stsWindows))*0
    ## Spike-field phase locking value
    sf_plv = Array{Float64}(length(f), sum(chNumUnits[spkChs]),length(stsWindows))*0
    ## Spike-field pairwise phase coherence
    sf_ppc0 = Array{Float64}(length(f), sum(chNumUnits[spkChs]),length(stsWindows))*0
    ## Number of total spikes
    nSpikesPerUnitPerWindow = Array{Int}(sum(chNumUnits[spkChs]), length(stsWindows))*0
    println(length(a)," windows, ", length(f), " frequencies, ", size(ch2ch,2)," channel-pairs will be used.")

    @time for win = 1:length(stsWindows), unit in 1:sum(chNumUnits)

        ## create mask for spiketriggeredspectrum and myPPC
        epochMask_sub = zeros(Int64,size(suLFPDup))
        winBegin = stsWindows[win]
        epochMask_sub[ winBegin: winBegin + stsStep,unit,:] += 1
        vecIndex_sub=find(suAllChns.*epochMask_sub.==1)
        if length(vecIndex_sub) > 0 
            nSpikesPerUnitPerWindow[unit,win] = length(vecIndex_sub)
            ## Syncrhony measures
            sts_sub = spiketriggeredspectrum(vecIndex_sub, vec(suLFPDup), -255:256, freqrange = fRange )

            sfc[:,unit,win] = pfcoherence(sts_sub,debias = false)
            sf_plv[:,unit,win] = pfplv(sts_sub)
            sf_ppc0[:,unit,win] = pfppc0(sts_sub) 
        end
       # println("win $win")
       # println(minimum(sf_ppc0[:,unit,win]))
    end

    ## Compute parameters and jackknife variances
    @time for win = 1:length(a)
        println("Computing multitaper for window no.",win,": (", a[win],"  ",b[win],")")
        (ps[:,:,win], co[:,:,win], ppc[:,:,win], plv[:,:,win], plv_jn,coh_jn, pli2unbiased[:,:,win] , 
            wpli[:,:,win], wpli2debiased[:,:,win])= multitaper(lfpAllChns[a[win]:b[win],:,:],
            (PowerSpectrum(), Coherence(),PPC(), PLV(), Jackknife(PLV()), Jackknife(Coherence()),
            PLI2Unbiased(), WPLI(),WPLI2Debiased() ), 1000,freqrange = fRange)

        (plv_bias[:,:,win], plv_variance[:,:,win]) = jackknife_bias_var(plv_jn...)
        (coh_bias[:,:,win], coh_variance[:,:,win]) = jackknife_bias_var(coh_jn...)
        # ps ~ nFreqs x nChannels 
        # xs, co ~ nFreqs x allpairs(nChannels)

    end

    # Write to output file
    @time begin
        spectralFile = matopen(spectralFileName,"w") 
        write(spectralFile,"ch2ch", ch2ch)
        write(spectralFile,"f",f)
        write(spectralFile,"suAllChnsTrlMean",suAllChnsTrlMean)
        write(spectralFile,"suAllChnsTrlStd",suAllChnsTrlStd)
        write(spectralFile,"powerSpectrum",ps)
        write(spectralFile,"coherence", co)
        write(spectralFile,"phaseLockingValue", plv)
        write(spectralFile,"pairwisePhaseConsistency", ppc)
        #write(spectralFile,"spikeTriggeredSpectrum", sts)
        #write(spectralFile,"spikeTriggeredSpectrum_phase", sts_phase)
        write(spectralFile,"spikeFieldWindows",collect(stsWindows))
        write(spectralFile,"spikeFieldCoherence", sfc)
        write(spectralFile,"spikeFieldPLV", sf_plv)
        write(spectralFile,"spikeFieldPPC", sf_ppc0)
        write(spectralFile,"sampleInd", sampleInd)
        write(spectralFile,"unitInd", unitInd)
        write(spectralFile,"trialInd", trialInd)
        write(spectralFile,"PLV_bias",plv_bias)
        write(spectralFile,"PLV_variance",plv_variance)
        write(spectralFile,"Coherence_bias",coh_bias)
        write(spectralFile,"Coherence_variance",coh_bias)
        write(spectralFile, "phaseLagIndex2unbiased", pli2unbiased)
        write(spectralFile, "weightedPhaseLagIndex", wpli)
        write(spectralFile, "weightedPhaseLagIndex2debiased", wpli2debiased)
        write(spectralFile, "nSpikesPerUnitPerWindow", nSpikesPerUnitPerWindow)
        close(spectralFile)
    end
    println("Saved $spectralFileName")
    
    ## Analysis params are written to meta file
    if tar == 6
        spectralMetaFileName = joinpath(file["dataDir"],file["session"],file["lfpDir"],
            join([file["lfpFileNamePrefix"], "spectral-metaInfo", file["fileFormat"]],""))

        spectralMetaFile = matopen(spectralMetaFileName,"w")

        write(spectralMetaFile,"spkChs", spkChs)
        write(spectralMetaFile,"chNumUnits",chNumUnits)
        write(spectralMetaFile,"nfft", nfft)  # number of samples to use for fft
        write(spectralMetaFile,"step", step)  # window of nfft points stepped every #step samples
        write(spectralMetaFile,"stsStep", stsStep)  # window of nfft points stepped every #step samples
        write(spectralMetaFile,"fRange", collect(fRange)) # number of frequency points to keep.")
        write(spectralMetaFile,"windowStartInd",collect(a))
        write(spectralMetaFile,"windowEndInd",collect(b))
        write(spectralMetaFile,"Epochs", Epochs)

        close(spectralMetaFile)
        println("Saved metafile: ", spectralMetaFileName)
    end
end

Processing target 1
Loading lfps for session: JS20131010, tar: 1
 19.583203 seconds (2.17 M allocations: 4.323 GB, 6.33% gc time)
Done loading data
233461 spikes total
50 windows, 80 frequencies, 4560 channel-pairs will be used.
1379.080396 seconds (52.33 M allocations: 1.512 TB, 9.43% gc time)
Computing multitaper for window no.1: (1  512)
Computing multitaper for window no.2: (51  562)
Computing multitaper for window no.3: (101  612)
Computing multitaper for window no.4: (151  662)
Computing multitaper for window no.5: (201  712)
Computing multitaper for window no.6: (251  762)
Computing multitaper for window no.7: (301  812)
Computing multitaper for window no.8: (351  862)
Computing multitaper for window no.9: (401  912)
Computing multitaper for window no.10: (451  962)
Computing multitaper for window no.11: (501  1012)
Computing multitaper for window no.12: (551  1062)
Computing multitaper for window no.13: (601  1112)
Computing multitaper for window no.14: (651  1162)
Computing mu

In [ ]:
## Computer canonical coherence

f = collect(frequencies(nfft, 1000)[1])
## indices of data segments, 3000 samples is hard coded here. Not ideal
winStart = 1:step:3000-nfft
winEnd = nfft:step:3000

## canonical coherence
CC = Array{Float64}(6,length(f),length(keys(groupPairs)),length(winStart))

for tar = 1:6

    println("Processing target ", tar)
    ## Data originally in trials x channels
    ## Shape data into samples x channels x trials for multitaper() (Synchrony.jl)

    ## Initialize data structure
    (a,b) = size(loadLFP(session,1,tar))
    lfpTemp = Array{Float64}(length(lfpChs),a,b)
    
    ## Load data into structure
    println("Loading lfps for session: $session, tar: $tar")
    @time for ch in collect(lfpChs)
        lfpTemp[ch,:,:] = loadLFP(session,ch,tar)
    end
    println("Done loading data")

    ## Permute data dimensions -> samples x channels x trials
    lfpAllChns = permutedims(lfpTemp,[3,1,2])
    
    ## Clear temp data
    lfpTemp = []

    for win = 1:length(winStart)


        println("Computing window no.",win,": (", winStart[win],"  ",winEnd[win],")")
        multitaperFFT=  mtfft(lfpAllChns[winStart[win]:winEnd[win],:,:], 1000)

        # multitaperFFT ~ nFreqs x nChannels x nTapers x nTrials

        
        for pIndex = 1:length(keys(groupPairs))

            ## Arrange data in channels x (# tapers x # trials) x n frequencies
            X = permutedims(multitaperFFT[:,channelMapping[groupPairs[pIndex][1]],:],[2,3,1])
            Y = permutedims(multitaperFFT[:,channelMapping[groupPairs[pIndex][2]],:],[2,3,1])

            ## Compute canonical covariance based on SVD algorithm
            for fIndex = 1:length(f)
                FX = svdfact(X[:,:,fIndex])
                FY = svdfact(Y[:,:,fIndex])
                Qxy = FX[:U]*ctranspose(FX[:V])*FY[:V]*ctranspose(FY[:U])
                FQ = svdfact(Qxy)
                CC[tar,fIndex,pIndex,win] = FQ[:S][1]
            end
        end
    end
    
end

In [ ]:
#@time begin
    ccFileName = joinpath(file["dataDir"],file["session"],file["lfpDir"],
join([file["lfpFileNamePrefix"], "allTar-canonicalCoherence_new", file["fileFormat"]],""))
    ccFile = matopen(ccFileName,"w")
    write(ccFile,"canonicalCoherence", CC)
    write(ccFile,"f",f)
    write(ccFile,"nfft",nfft)
    write(ccFile,"step",step)
    write(ccFile,"windowStartInd",collect(winStart))
    write(ccFile,"windowEndInd",collect(winEnd))
    groupPairsName = join([join(groupPairs[i],"-") for i in 1:length(keys(groupPairs))], " ")
    write(ccFile,"groupPairs",groupPairsName)
    write(ccFile,"channelMapping", channelMapping)
    close(ccFile)
#end
println("Saved $ccFile")